## Import The relevant packages

In [1]:
import numpy as np
import tensorflow as tf

## Data

In [2]:
# Defining Values

DATASET_SIZE = 70000
TRAIN_RATIO = 0.8
VALIDATION_RATIO = 0.1
TEST_RATIO = 0.1

# Loading MNIST dataset and shuffling 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

X = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])

BUFFER_SIZE = 1000

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(BUFFER_SIZE)

# Scaling the data by dividing each and every value by 255.
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


# splitting Train, Validation, and Test sets and Scaling them using .map() function

train_dataset = (dataset.take(int(TRAIN_RATIO*DATASET_SIZE))).map(scale)
validation_dataset = (dataset.skip(int(TRAIN_RATIO*DATASET_SIZE)).take(int(VALIDATION_RATIO*DATASET_SIZE))).map(scale)
test_dataset = (dataset.skip(int(TRAIN_RATIO*DATASET_SIZE)).skip(int(VALIDATION_RATIO*DATASET_SIZE))).map(scale)

### Creating batches for mini-batch gredient descent

In [7]:
# Creating batches it the Train and Validation datasets to use mini-batch gradient descent

Batch_size = 10

train_data = train_dataset.batch(Batch_size)

num_validation_samples = tf.cast(VALIDATION_RATIO * DATASET_SIZE, tf.int64)
validation_data = validation_dataset.batch(num_validation_samples)

num_test_samples = tf.cast(TEST_RATIO * DATASET_SIZE, tf.int64)
test_data = test_dataset.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model



### outline the model

In [8]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            # flatten input to convert into a vector
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
    
                            # using .Dense() method to calculate the dot product of weights and inputs and add biases. it can also be used to apply activation function.
                            tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation= 'tanh'),
                            tf.keras.layers.Dense(output_size, activation= 'softmax') 
                            ])



### Choose the optimizer and loss function

In [9]:
custom_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training 

In [10]:
num_epochs = 5

model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
5600/5600 - 5s - loss: 0.3827 - accuracy: 0.8947 - val_loss: 0.2149 - val_accuracy: 0.9370 - 5s/epoch - 957us/step
Epoch 2/5
5600/5600 - 5s - loss: 0.1894 - accuracy: 0.9444 - val_loss: 0.1550 - val_accuracy: 0.9556 - 5s/epoch - 882us/step
Epoch 3/5
5600/5600 - 5s - loss: 0.1361 - accuracy: 0.9597 - val_loss: 0.1204 - val_accuracy: 0.9663 - 5s/epoch - 902us/step
Epoch 4/5
5600/5600 - 5s - loss: 0.1052 - accuracy: 0.9696 - val_loss: 0.1076 - val_accuracy: 0.9691 - 5s/epoch - 922us/step
Epoch 5/5
5600/5600 - 5s - loss: 0.0860 - accuracy: 0.9747 - val_loss: 0.0953 - val_accuracy: 0.9737 - 5s/epoch - 894us/step


### Test the model

In [11]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 171ms/step - loss: 0.0792 - accuracy: 0.9764


In [12]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))

Test loss: 0.08. Test accuracy: 97.64%
